In [2]:
!pip install langchain-community
!pip install groq
!pip install langchain-groq
!pip install faiss-cpu
!pip install langchain sentence-transformers torch
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from groq import Groq
from sentence_transformers import SentenceTransformer
import torch
from langchain_groq import ChatGroq

#  Secure API Key Handling
os.environ["GROQ_API_KEY"] = "groq api key"

#  Load Pretrained Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")

#  CV Text Data
cv_text = """
Name: Md Shafiqul Islam
Contact: [Your Email] | [Your Phone Number] | [LinkedIn Profile]
details cv.....pp
"""

#  Step 1: Text Chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", ", "]
)
cv_chunks = text_splitter.split_text(cv_text)

#  Step 2: Convert Chunks into Vector Embeddings
vector_store = FAISS.from_texts(cv_chunks, embedding_model)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

#  Step 3: Initialize Groq API
llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model="llama-3.1-8b-instant",
    temperature=0.7
)
#  Step 4: Initialize Memory First
memory = ConversationBufferMemory(memory_key="chat_history", output_key="answer", return_messages=True)

#  Step 5: Setup Conversational Retrieval Chain
from langchain.chains import ConversationalRetrievalChain

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True  # Optional: Returns source text used for answers
)

#  Step 6: Interactive Chatbot
def cv_assistant():
    print("\n📝 CV Assistant is ready! Ask me about your CV (type 'exit' to quit).")

    while True:
        question = input("\n❓ Your Question: ")
        if question.lower() == "exit":
            print("\n👋 Goodbye!")
            break

        response = qa_chain.invoke({"question": question})
        print(response)
        answer = response["answer"]
        print(f"\n💬 Answer: {answer}")

#  Start the Assistant
if __name__ == "__main__":
    cv_assistant()



📝 CV Assistant is ready! Ask me about your CV (type 'exit' to quit).


KeyboardInterrupt: Interrupted by user